Title: Ranking Results with Window Functions
Slug: sql/ranking-results-with-window-functions
Category: SQL
Tags: SELECT, FROM, LIMIT, WHERE, OVER, PARTITION BY, ORDER BY, DESC, rank
Date: 2017-08-15
Modified: 2017-10-23

#### Load ipython-sql extension

In [1]:
# The 2 lines below prevent an error message from being displayed when we run %load_ext sql
import warnings
warnings.filterwarnings('ignore')

%load_ext sql
%config SqlMagic.feedback = False

#### Connect to the database

In [2]:
%%sql

postgresql://localhost/dvdrental

'Connected: None@dvdrental'

#### Ranking results
Using the `OVER` clause allows us to run a function on the results returned from the database. In this example we rank films from longest to shortest at each rental rate.

In [3]:
%%sql

SELECT
    f.film_id
    , f.title
    , f.rental_rate
    , f.length
    , rank() OVER(PARTITION BY f.rental_rate ORDER BY f.length DESC) as rk
FROM
    film f
LIMIT
    5

film_id,title,rental_rate,length,rk
813,Smoochy Control,0.99,184,1
886,Theory Mermaid,0.99,184,1
821,Sorority Queen,0.99,184,1
996,Young Language,0.99,183,4
128,Catch Amistad,0.99,183,4


Note that since 3 films are tied for first place, the rank given to the next longest films is 4. If we wanted the next rank to be 2 instead, we could use `dense_rank` in place of `rank`.

#### Working with our rankings
Once you've made the query above, we can use it as a [Common Table Expression](../sql/common-table-expressions.html) to filter it. For example, let's try to find the 3 longest films at each price point.

In [4]:
%%sql

WITH film_cte AS (
    SELECT
        f.film_id
        , f.title
        , f.rental_rate
        , f.length
        , rank() OVER(PARTITION BY f.rental_rate ORDER BY f.length DESC) as rk
    FROM
        film f
)

SELECT
    *
FROM
    film_cte fc
WHERE
    fc.rk < 4

film_id,title,rental_rate,length,rk
813,Smoochy Control,0.99,184,1
886,Theory Mermaid,0.99,184,1
821,Sorority Queen,0.99,184,1
690,Pond Seattle,2.99,185,1
872,Sweet Brotherhood,2.99,185,1
991,Worst Banger,2.99,185,1
349,Gangs Pride,2.99,185,1
609,Muscle Bright,2.99,185,1
426,Home Pity,4.99,185,1
817,Soldiers Evolution,4.99,185,1


#### Tailoring our results
Hmm, it looks like more than three films are tied in the £2.99 and £4.99 price bands. If we want to avoid this, one option is to use `row_number`, specifying how to sort the results.

Now we'll return the 3 longest films at each rental rate in alphabetical order.

In [5]:
%%sql

WITH film_ranked AS (
    SELECT
        f.film_id
        , f.title
        , f.rental_rate
        , f.length
        , row_number() OVER(PARTITION BY f.rental_rate ORDER BY f.length DESC, f.title ASC) as rk
    FROM
        film f
)

SELECT
    *
FROM
    film_ranked fr
WHERE
    fr.rk < 4

film_id,title,rental_rate,length,rk
813,Smoochy Control,0.99,184,1
821,Sorority Queen,0.99,184,2
886,Theory Mermaid,0.99,184,3
349,Gangs Pride,2.99,185,1
609,Muscle Bright,2.99,185,2
690,Pond Seattle,2.99,185,3
141,Chicago North,4.99,185,1
182,Control Anthem,4.99,185,2
212,Darn Forrester,4.99,185,3


There are lots of other window functions that can also be used here. [Check the Postgres docs](https://www.postgresql.org/docs/current/static/functions-window.html) for full details.